<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/Preprocessing_CrowdFlower_using_Ekphrasis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initialization

In [ ]:
!pip3 install ekphrasis
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import numpy as np
import pandas as pd
import re, sys, os, csv
import io

In [3]:
import nltk
from nltk.corpus import words
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [4]:
from nltk.metrics.distance import (edit_distance,jaccard_distance)
from nltk.util import ngrams

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


#Methods for Cleaning

In [6]:
emoticons_additional = {
    '(^・^)': '<happy>', ':‑c': '<sad>', '=‑d': '<happy>', ":'‑)": '<happy>', ':‑d': '<laugh>',
    ':‑(': '<sad>', ';‑)': '<happy>', ':‑)': '<happy>', ':\\/': '<sad>', 'd=<': '<annoyed>',
    ':‑/': '<annoyed>', ';‑]': '<happy>', '(^�^)': '<happy>', 'angru': 'angry', "d‑':":
        '<annoyed>', ":'‑(": '<sad>', ":‑[": '<annoyed>', '(�?�)': '<happy>', 'x‑d': '<laugh>',
}


In [7]:
text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
               'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
              'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter",
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter",
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=True,  # spell correction for elongated words
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons, emoticons_additional]
)

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [15]:
def tokenize(text):
    text = " ".join(text_processor.pre_process_doc(text))
    return text

In [16]:
def tweet_label(label):
    if label == "empty":return 0 # neutral
    elif label == "sadness":return 2 # sad
    elif label == "enthusiasm":return 1 # happy
    elif label == "neutral":return 0 # neutral
    elif label == "worry":return 2 # sad
    elif label == "surprise":return 1 # happy
    elif label == "love":return 1 # happy
    elif label == "fun":return 1 # happy
    elif label == "hate":return 3 #hate
    elif label == "happiness":return 1 # happy
    elif label == "boredom":return 0 # neutral
    elif label == "relief":return 1 # happy
    elif label == "anger":return 3 #anger

#cleaning Process

In [9]:
crowdFlower = pd.read_csv('/content/gdrive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower.csv',names = ['emotion','tweets'],skiprows = 1,header=None)

In [10]:
crowdFlower.head()

,emotion,tweets
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [11]:
try :
  crowdFlower.drop(['tweet_id','author'],axis=1,inplace=True)
except : 
  print("already removed")
crowdFlower.head()

already removed


,emotion,tweets
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [12]:
crowdFlower.to_csv('/content/gdrive/MyDrive/InfluenceAnalysis/crowdFlower/crowdFlower.csv',index=None)

In [17]:
crowdFlower['tweets'] = crowdFlower['tweets'].apply(lambda tweet : tokenize(tweet))
crowdFlower['emotion'] = crowdFlower['emotion'].apply(lambda label : tweet_label(label))

In [18]:
crowdFlower.head()

,emotion,tweets
0,0,<user> i know i was listenin to bad habit earl...
1,2,layin n bed with a headache ugh <elongated> . ...
2,2,funeral ceremony . <repeated> gloomy friday . ...
3,1,wants to hang out with friends <allcaps> soon ...
4,0,<user> we want to trade with someone who has h...


In [19]:
crowdFlower['emotion'].value_counts()

1    15299
2    13624
0     9644
3     1433
Name: emotion, dtype: int64

In [20]:
crowdFlower.to_csv('/content/gdrive/MyDrive/InfluenceAnalysis/crowdFlower/cF_clean_Ekphrasis_combined.csv')

#Spelling Correction Methods

In [ ]:
def jaccard(word,gram_number):
  
  try:
    if dictionary[word]==1:
      return word
  except KeyError:
    spellings = spelling_series[spelling_series.str.startswith(word[:2])]

    distances = []
    if len(word) >=3: 
      distances = [(jaccard_distance(set(ngrams(word,3)),set(ngrams(x,3))),x) for x in spellings]
    elif len(word) == 2:
      distances = [(jaccard_distance(set(ngrams(word,2)),set(ngrams(x,2))),x) for x in spellings]
    
    closet = ("0",word)
    if len(distances)!=0:
      closet = min(distances)
    return closet[1]

In [ ]:
def correct_spelling(word):
  return str(jaccard(word,3))


In [ ]:
def correct_spelling_tweet(tweet):

  words = tweet.split()
  for i in range(len(words)):
    words[i] = correct_spelling(words[i])

  return " ".join(words)

In [ ]:
vocab_glove = {}
spellings = []
dictionary = {}

In [ ]:
with open("/content/gdrive/MyDrive/InfluenceAnalysis/glove/glove.twitter.27B.200d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      dictionary[word] = 1
      spellings.append(word)
      vocab_glove[word] = vector

In [ ]:
spelling_series = pd.Series(spellings)
print(spelling_series)

0            <user>
1                 .
2                 :
3                rt
4                 ,
             ...   
1193509     ﾊﾞｲﾊﾞｰｲ
1193510        ﾊﾟﾝﾁ
1193511       ﾔﾒﾀﾏｴ
1193512       ﾖｲｼｮｯ
1193513    ﾟﾟﾟｵﾔｽﾐｰ
Length: 1193514, dtype: object


In [ ]:
 crowdFlower['tweets'] = crowdFlower['tweets'].apply(lambda x : correct_spelling_tweet(x))

NameError: ignored